In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [2]:
batch_size = 4
seq_len = 100
feat_dim = 20
emb_dim = 512
num_classes = 10
rank = 2

input_ = torch.rand(batch_size, seq_len, feat_dim)
print(input_.shape)

torch.Size([4, 100, 20])


In [3]:
class MatrixFactorisation(nn.Module):
    
    def __init__(self, seq_len, feat_dim, emb_dim, rank, num_classes):
        super(MatrixFactorisation, self).__init__()
        for k, v in locals().items():
            if k != "self":
                setattr(self, k, v)
                
        self.linear1 = nn.Linear(feat_dim, emb_dim)
        self.linear2 = nn.Linear(seq_len, num_classes)
        self.ff = nn.Linear(emb_dim, num_classes)
                
    def forward(self, input_):
        matrix = self.linear1(input_)
        matrix = self.linear2(matrix.transpose(1, 2))
        embeds = [self.ff(matrix[:, :, idx]) for idx in range(rank)]
        embeds = [nn.Softmax(dim=1)(embed) for embed in embeds]
        output = torch.mean(torch.stack(embeds, dim=1), dim=1)
        return output

In [4]:
model = MatrixFactorisation(seq_len, feat_dim, emb_dim, rank, num_classes)
output_ = model(input_)

In [5]:
output_.shape

torch.Size([4, 10])